Resource: https://www.wikiart.org/

* 爬風格： https://www.wikiart.org/en/paintings-by-style

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import os
from tqdm import tqdm
import urllib.request
from msedge.selenium_tools import Edge, EdgeOptions
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities


In [2]:
# Primitivism 在 wikiart title 是 na-ve-art-primitivism
# Mannerism 在 wikiart title 是 mannerism-late-renaissance

BASE_URL = 'https://www.wikiart.org/en/paintings-by-style/'
def convert_url(title):
    # print(title)
    if(title == 'Primitivism'):
        return BASE_URL + 'na-ve-art-primitivism'
    elif (title == 'Mannerism'):
        return BASE_URL + 'mannerism-late-renaissance'
    else:
        # 兩字節的 title 空格換成 -
        if(len(title.split(' ')) == 2):
            title = title.replace(' ', '-')
        return BASE_URL + title

In [3]:
'''
網站圖畫數量太少，所以不爬的種類：
    Neoplasticism  183
    Byzantine 256
    Renaissance 33
    Suprematism 136
'''

artists = pd.read_csv('../artists.csv',usecols=['name', 'genre','paintings'])

for i in range(len(artists)):
    temp_data = artists.loc[i,'genre']
    temp_data = temp_data.split(',')
    if(len(temp_data) <= 1):
        pass
    else:
        artists.drop(index=i, inplace=True)
artists_crawl_df = artists.groupby('genre').sum().sort_values(by='paintings',ascending=False)
drop_genre = ['Neoplasticism', 'Byzantine Art', 'Renaissance','Suprematism']
artists_crawl_df = artists_crawl_df[ ~artists_crawl_df.index.isin(drop_genre)].reset_index()
artists_crawl_df['crawl_url'] = artists_crawl_df['genre'].apply(convert_url)
artists_crawl_df

,genre,paintings,crawl_url
0,Impressionism,1370,https://www.wikiart.org/en/paintings-by-style/...
1,Post-Impressionism,1048,https://www.wikiart.org/en/paintings-by-style/...
2,Northern Renaissance,680,https://www.wikiart.org/en/paintings-by-style/...
3,Baroque,586,https://www.wikiart.org/en/paintings-by-style/...
4,Cubism,439,https://www.wikiart.org/en/paintings-by-style/...
5,Romanticism,388,https://www.wikiart.org/en/paintings-by-style/...
6,Primitivism,309,https://www.wikiart.org/en/paintings-by-style/...
7,High Renaissance,301,https://www.wikiart.org/en/paintings-by-style/...
8,Surrealism,241,https://www.wikiart.org/en/paintings-by-style/...
9,Expressionism,193,https://www.wikiart.org/en/paintings-by-style/...


In [30]:
artists_crawl_df.to_csv('genre.csv',index=False)

In [4]:
# 先自己創立 ROOT_FOLDER
ROOT_FOLDER_PATH = r'C:\Users\Stephanie Chung\Documents\NTUIM 2022 Fall\深度學習實作應用\Final Project\Wikiart_webcrawl\genre_data'

for i in range(0,17):
    genre = artists_crawl_df.loc[i,'genre']
    url = artists_crawl_df.loc[i,'crawl_url']
    # print(url)
    
    # 若不存在該種類則建立此資料夾
    if not os.path.exists(os.path.join(ROOT_FOLDER_PATH, genre)):
        os.makedirs(os.path.join(ROOT_FOLDER_PATH, genre))

## 開始爬蟲

In [25]:
url = artists_crawl_df.loc[16,'crawl_url']
store_folder = artists_crawl_df.loc[16,'genre']
print(store_folder)
options = EdgeOptions()
options.use_chromium = True
#options.add_argument("-inprivate")
options.binary_location = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe"
capabilities = DesiredCapabilities.EDGE

driver = Edge(executable_path = r"C:\Users\Stephanie Chung\Desktop\msedgedriver.exe", options = options,desired_capabilities=capabilities) # Modify the path here...

driver.get(url) 

for num in range(0,8):
    check_height = driver.execute_script("return document.body.scrollHeight;") 
    current_height = driver.execute_script("return window.pageYOffset + window.innerHeight")
    print(num)
    # print(height)
    # 因為 lazy loading，所以要慢慢滑讀資料
    for i in range(int(current_height),check_height):
        driver.execute_script(f"window.scrollTo({current_height},{i})")

    # download the image
    soup = BeautifulSoup(driver.page_source, "html.parser")
    data = soup.find(class_="masonry-content").find('ul').find_all('li')
    for img in data:
        img_src = img.find('img').get('src')
        normal_img_src = img_src.split('!')[0]
        save_src = normal_img_src.split('/')[-1]
        save_path = fr'./genre_data/{store_folder}/' + str(save_src)
        # urllib.request.urlretrieve(normal_img_src, save_path)
        # print(path)
    driver.find_elements(By.XPATH, "//a[@class='masonry-load-more-button']")[0].click()
    time.sleep(10)

Abstract Expressionism


C:\Users\STEPHA~1\AppData\Local\Temp/ipykernel_14844/3171390948.py:10: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(executable_path = r"C:\Users\Stephanie Chung\Desktop\msedgedriver.exe", options = options,desired_capabilities=capabilities) # Modify the path here...


In [26]:

soup = BeautifulSoup(driver.page_source, "html.parser")
data = soup.find(class_="masonry-content").find('ul').find_all('li')


In [27]:
len(data)

360

In [28]:
cnt = 0
for img in tqdm(range(len(data))):
    img_src = data[img].find('img').get('src')
    normal_img_src = img_src.split('!')[0]
    save_src = normal_img_src.split('/')[-1]
    save_path = fr'./genre_data/{store_folder}/{cnt}.jpg'
    try:
        urllib.request.urlretrieve(normal_img_src, save_path)
        cnt += 1
    except:
        print('error: ',img)
        continue
        # print(path)

100%|██████████| 360/360 [12:54<00:00,  2.15s/it]
